# Plots and maps
In this notebook we show how to create plots from the data and the 2D fields.

There are different Julia packages allowing users to display their data.
- [`PyPlot`](https://github.com/JuliaPy/PyPlot.jl) is a Julia interface to the [`Matplotlib`](https://matplotlib.org/) (Python) plotting library, which uses the `PyCall` package to call the Python code.
- [`Makie`](https://docs.makie.org), a data visualization ecosystem written in Julia.

🗺️ For the creation of maps:
- `PyPlot` can work with modules such as [`cartopy`](https://scitools.org.uk/cartopy/docs/latest/) or [`Basemap`](https://matplotlib.org/basemap/stable/).
- `Makie` can be used in combination with [`GeoMakie`](https://geo.makie.or), which allows one to manage different coordinate reference systems (CRS).

<div class="alert alert-block alert-warning"> 
  ⚠️ The use of `PyPlot` (+ <code>cartopy</code> or <code>Basemap</code>) can lead to installation errors which are not always easy to solve. <br> 
This is why we will focus more on the creation of figures with <code>Makie</code>.
</div>

## Packages

In [ ]:
using Dates
using Makie, CairoMakie, GeoMakie
using DIVAnd
using NCDatasets
using ColorSchemes
include("../config.jl")

[ Info: Precompiling Makie [ee78f7c6-11fb-53f2-987a-cfe4a2b5a57a] (cache misses: incompatible header (18))
[ Info: Precompiling CairoMakie [13f3f980-e62b-5c42-98c6-ff1f3baf88f0] (cache misses: incompatible header (18))
[ Info: Precompiling GeoMakie [db073c08-6b98-4ee5-b6a4-5efafb3259c6] (cache misses: wrong dep version loaded (2), incompatible header (18))
[ Info: Precompiling DIVAnd [efc8151c-67de-5a8f-9a35-d8f54746ae9d] (cache misses: incompatible header (18))


## Bathymetry and mask
### Download and load file

In [ ]:
download_check(gebco08file, gebco08fileURL);
bx, by, b = extract_bath(gebco08file, true, 9.0:15.0, 54.0:60.0);
lonmask, latmask, mask = load_mask(gebco08file, true, 9.0:0.1:15.0, 54.0:0.1:60.0, [0.0])

### Plots
Here we will show the results obtained with `Makie` and with `GeoMakie` to show the difference.     
Then the plots will be only produced with `GeoMakie`
#### With Makie
We represent the bathymetry and specify
- the colormap (`Reverse(ColorSchemes.delta)`)
- the limits (`colorrange=[-500, 500]`).      

In addition, the contour corresponding to the surface (`levels=[0.]`) is added to show the coastline.     

A list of available colormaps can be found at https://docs.makie.org/dev/explanations/colors.

In [ ]:
topocmap = Reverse(ColorSchemes.delta);

In [ ]:
f = Figure()
ax = Axis(f[1, 1], title = "Bathymetry with Makie")
hm = heatmap!(ax, bx, by, b, colormap = topocmap, colorrange = [-500, 500])
contour!(ax, bx, by, b, levels = [0.0], color = :black)
Colorbar(f[1, 2], hm, label = "(m)")
f

#### With GeoMakie
The main difference with the previous plot is that we use a `GeoAxis` (instead of an `Axis`),      
for which we can define the coordinate reference system with the argument `dest = "+proj=merc"`.     
The available projections are those listed at https://proj.org/en/stable/operations/projections/index.html.      
The coastline can be added with `lines!(ax, GeoMakie.coastlines())`.

In [ ]:
f = Figure()
ax = GeoAxis(f[1, 1], title = "Bathymetry with Makie", dest = "+proj=hatano")
hm = heatmap!(ax, bx, by, b, colormap = topocmap, colorrange = [-500, 500])
contour!(ax, bx, by, b, levels = [0.0, 1], color = :black, label = "From GEBCO topography")
lines!(ax, GeoMakie.coastlines(), color = :red, label = "From GeoMakie")
Colorbar(f[1, 2], hm, label = "(m)")
f[1, 3] = Legend(f, ax, "Coastline")
xlims!(9.0, 15.0)
ylims!(54.0, 60.0)
f

### Land-sea mask
The mask contains `0`s and `1`s, we can display it using the function `heatmap` with for instance a black-and-white colormap (`:binary`).

In [ ]:
f = Figure()
ax = GeoAxis(f[1, 1], title = "Bathymetry with Makie", dest = "+proj=merc")
hm = heatmap!(ax, lonmask, latmask, mask[:, :, 1], colormap = Reverse(:binary))
f

## Observations
### Load file
We work with salinity data in the Baltic Sea.

In [ ]:
download_check(balticfile, balticfileURL)
obsvalue, obslon, obslat, obsdepth, obstime, obsids =
    loadobs(Float64, balticfile, "salinity");

In [ ]:
sel = findall((obslon .<= 15.0) .& (obstime .>= DateTime(2011, 1, 1)));
@info(length(sel))

### Positions only
We can use either `Makie` (left panel) or `GeoMakie` (right panel) for the plots.        

In [ ]:
f = Figure()
ax1 = Axis(f[1, 1])
plot!(ax1, obslon[sel], obslat[sel], color = :blue, markersize = 2)
ax2 = GeoAxis(f[1, 2], dest = "+proj=merc")
plot!(ax2, obslon[sel], obslat[sel], color = :blue, markersize = 2)
f

### Scatter plot
The color depends on the data value

In [ ]:
f = Figure()
ax1 = Axis(f[1, 1])
plot!(ax1, obslon[sel], obslat[sel], color = obsvalue[sel], markersize = 5)
ax2 = GeoAxis(f[1, 2], dest = "+proj=merc")
plot!(ax2, obslon[sel], obslat[sel], color = obsvalue[sel], markersize = 5)
f

## Horizontal maps
There are different functions that allow us to represent 2-dimensional fields.
- As we have seen with the topography, we can use [`heatmap`](https://docs.makie.org/stable/reference/plots/heatmap), which representes the values as a collection of colored rectangles;
- The function [`surface`](https://docs.makie.org/v0.21/reference/plots/surface) works similarly but with different options and rendering.

We will illustrate both functions with the salinity in the Black Sea.

<div class="alert alert-block alert-warning">
⚠️ <code>heatmap</code> requires the coordinates to be defined as vectors, not matrices.
</div>

In [ ]:
download_check(outputsalinity, outputsalinityURL)

ds = NCDataset(outputsalinity)
lon = ds["lon"][:]
lat = ds["lat"][:]
salinity = ds["Salinity"][:, :, 1, 1]
close(ds)

### Heatmap
With `heatmap` the grid is visible as no interpolation if performed by default (optional parameter `interpolate=false`).

In [ ]:
f = Figure()
ax = GeoAxis(
    f[1, 1],
    title = "Interpolated salinity field",
    dest = "+proj=merc",
    xticks = 27:2:42,
)
hm = heatmap!(
    ax,
    lon,
    lat,
    salinity,
    colormap = ColorSchemes.haline,
    colorrange = [16.0, 18.5],
)
Colorbar(f[1, 2], hm, label = "S")
f

### Surface
The `surface` function has a different behaviour by default, as it performs interpolation and also shading effects. This can be avoided using the following options:    
- `shading=NoShading`
- `interpolate=false`

even if the latter doesn't seem to affect the plot. 

In [ ]:
f = Figure()
ax = GeoAxis(
    f[1, 1],
    title = "Interpolated salinity field",
    dest = "+proj=merc",
    xticks = 27:2:42,
)
hm = surface!(
    ax,
    lon,
    lat,
    salinity,
    colormap = ColorSchemes.haline,
    colorrange = [16.0, 18.5],
    shading = NoShading,
    interpolate = false,
)
Colorbar(f[1, 2], hm, label = "S")
f

### Vertical section
We extract the salinity at the latitude 42°N.

In [ ]:
indexlat = findfirst(lat .== 42.0)

ds = NCDataset(outputsalinity)
lon = ds["lon"][:]
lat_s = ds["lat"][indexlat]
salinity_s = ds["Salinity"][:, indexlat, :, 1]
depth = ds["depth"][:]
close(ds)

In [ ]:
f = Figure()
ax = Axis(
    f[1, 1],
    xlabel = "Longitude (°E)",
    ylabel = "Depth (m)",
    title = "Vertical section at $(lat_s)°N",
)
heatmap!(ax, lon, -depth, salinity_s)
f

## Velocity field

In [ ]:
download_check(velfilename, velfilenameURL)

In [ ]:
nc = NCDataset(velfilename)
uvlon = nc["lon"][:]
uvlat = nc["lat"][:]
uvel = coalesce.(nc["vozocrtx"][:, :, 1, 1], NaN)
vvel = coalesce.(nc["vomecrty"][:, :, 1, 1], NaN)
close(nc)

### Simple arrows

In [ ]:
fig = Figure();
r = 2;
ga = Axis(
    fig[1, 1];
    # dest = "+proj=merc",
    title = "Velocity field",
    xticks = 8:4:20,
    yticks = 38:2:46,
)
arrows!(
    ga,
    uvlon[1:r:end],
    uvlat[1:r:end],
    uvel[1:r:end, 1:r:end, 1],
    vvel[1:r:end, 1:r:end],
    arrowsize = 4,
    lengthscale = 4,
)
# Reference vector
arrows!(ga, [18.0], [44.5], [0.25], [0.0], arrowsize = 4, lengthscale = 4)
text!(ga, 18, 44.6, text = "0.25 m/s")
fig

### Colored arrows
We use the velocity norm to specify the color.      
To limit the density of vectors, we decrease the resolution of a factor `r` (`=2` by the value can be changed).

In [ ]:
r = 2;
velnorm = vec(
    sqrt.(
        uvel[1:r:end, 1:r:end] .* uvel[1:r:end, 1:r:end] .+
        vvel[1:r:end, 1:r:end] .* vvel[1:r:end, 1:r:end]
    ),
);

The color of the arrow and of the line can be set seperately with the options `arrowcolor` and `linecolor`:

In [ ]:
fig = Figure();
r = 2;
ga = Axis(
    fig[1, 1];
    # dest = "+proj=merc",
    title = "Velocity field",
    xticks = 8:4:20,
    yticks = 38:2:46,
)
arrows!(
    ga,
    uvlon[1:r:end],
    uvlat[1:r:end],
    uvel[1:r:end, 1:r:end, 1],
    vvel[1:r:end, 1:r:end],
    arrowsize = 4,
    lengthscale = 2,
    arrowcolor = velnorm,
    linecolor = velnorm,
)
fig